In [9]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2

In [10]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None, dropout_rate=0.1):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels, dropout_rate=0.1):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels, dropout_rate=dropout_rate)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True, dropout_rate=0.1):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2, dropout_rate=dropout_rate)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels, dropout_rate=dropout_rate)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [11]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False, dropout_rate=0.1):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64, dropout_rate=dropout_rate))
        self.down1 = (Down(64, 128, dropout_rate=dropout_rate))
        self.down2 = (Down(128, 256, dropout_rate=dropout_rate))
        self.down3 = (Down(256, 512, dropout_rate=dropout_rate))
        factor = 2 if bilinear else 1
        self.down4 = (Down(512, 1024 // factor, dropout_rate=dropout_rate))
        self.up1 = (Up(1024, 512 // factor, bilinear, dropout_rate=dropout_rate))
        self.up2 = (Up(512, 256 // factor, bilinear, dropout_rate=dropout_rate))
        self.up3 = (Up(256, 128 // factor, bilinear, dropout_rate=dropout_rate))
        self.up4 = (Up(128, 64, bilinear, dropout_rate=dropout_rate))
        self.outc = (OutConv(64, n_classes))

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [12]:
PATCH_SIZE = (256, 256)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
unet = UNet(1, 2, bilinear=False, dropout_rate=0.1).to(DEVICE)

unet = nn.DataParallel(unet)
unet.load_state_dict(torch.load("/kaggle/input/best_model/pytorch/default/1/best_model.pth", map_location=torch.device(DEVICE)))

/tmp/ipykernel_31/2631330684.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet.load_state_dict(torch.load("/kaggle/input/best_model/pytorch/default/1/best_model.pth",

<All keys matched successfully>

In [14]:
def get_nodule(model, image):
    model.eval()
    image = transforms.Resize(PATCH_SIZE, antialias=True)(image)
    image = transforms.ToTensor()(image)               # [1, H, W]

    with torch.no_grad():
        output = model(image.unsqueeze(0).to(DEVICE)).cpu()
    pred_mask = torch.argmax(output, dim=1).float()
    
    mask_np = pred_mask.squeeze().numpy().astype(np.uint8)
    kernel  = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opened  = cv2.morphologyEx(mask_np, cv2.MORPH_OPEN, kernel)
    dilated = cv2.morphologyEx(opened, cv2.MORPH_DILATE, kernel)
    binary_mask = torch.from_numpy(dilated).unsqueeze(0)

    coords = cv2.findNonZero(dilated)
    if coords is None:  # If no coordinates are found (empty mask)
        print("Warning: Skipping image due to no valid coordinates.")
        return None  # Skip this image
        
    x, y, w, h = cv2.boundingRect(coords)
    cropped_img  = image[:, y : y+h, x : x+w]
    cropped_mask = binary_mask[:, y : y+h, x : x+w]
    
    if cropped_img.shape[-2] == 0 or cropped_img.shape[-1] == 0:
        print("Warning: Skipping image due to empty crop.")
        return None

    batch_img  = cropped_img.unsqueeze(0)
    batch_mask = cropped_mask.unsqueeze(0)
    resized_img  = F.interpolate(batch_img, size=(300,300), mode='bilinear',    align_corners=False).squeeze(0)
    resized_mask = F.interpolate(batch_mask, size=(300,300), mode='nearest').squeeze(0)
    masked_image = resized_img * resized_mask
    return masked_image

In [15]:
import pandas as pd
from sklearn.utils import resample

df = pd.read_csv('/kaggle/input/data/Data_Entry_2017.csv')

nodule_mask     = df['Finding Labels'].apply(lambda labels: 'Nodule' in labels.split('|'))
other_mask      = ~nodule_mask

nodule_df       = df[nodule_mask]
second_df       = df[other_mask]

In [16]:
import os
import zipfile
import pandas as pd
from PIL import Image
import os
import pandas as pd
import zipfile
from torchvision import transforms
from PIL import Image

source_base_dir = '/kaggle/input/data/'

OUT_BASE = "/kaggle/working/processed_lungs"
os.makedirs(os.path.join(OUT_BASE, "nodule"),    exist_ok=True)
os.makedirs(os.path.join(OUT_BASE, "non_nodule"), exist_ok=True)

folder_ranges = [
    (1335, 6, 'images_001'),
    (3923, 13, 'images_002'),
    (6585, 6, 'images_003'),
    (9232, 3, 'images_004'),
    (11558, 7, 'images_005'),
    (13774, 25, 'images_006'),
    (16051, 9, 'images_007'),
    (18387, 34, 'images_008'),
    (20945, 49, 'images_009'),
    (24717, 0, 'images_010'),
    (28173, 2, 'images_011'),
    (30805, 0, 'images_012')
]
records = []
for cohort_df, label_value in [(nodule_df, 1.0), (second_df, 0.0)]:
    subfolder = "nodule" if label_value == 1.0 else "non_nodule"
    out_dir   = os.path.join(OUT_BASE, subfolder)
    
    for _, row in cohort_df.iterrows():
        image_filename = row['Image Index']
        
        base_name = os.path.splitext(image_filename)[0]
        part1_str, part2_str = base_name.split('_')
        part1, part2 = int(part1_str), int(part2_str)

        source_folder = None
        for f_part1, f_part2, f_name in folder_ranges:
            if part1 < f_part1 or (part1 == f_part1 and part2 <= f_part2):
                source_folder = f_name
                break
        if not source_folder:
            continue

        img_path = os.path.join(source_base_dir,
                                source_folder,
                                'images',
                                image_filename)

        image = Image.open(img_path).convert('L')
        image = get_nodule(unet, image)
        
        if image is not None:
            pil_img = transforms.ToPILImage()(image)
            save_name = f"{base_name}_lung.png"
            save_path = os.path.join(out_dir, save_name)
            pil_img.save(save_path)
            
            records.append({
                "filename": save_name,
                "label":    int(label_value)
            })
        else:
            print("Warning: Skipping image due to invalid or empty result from get_nodule.")


df_out = pd.DataFrame(records)
csv_path = os.path.join(OUT_BASE, "labels.csv")
df_out.to_csv(csv_path, index=False)
print(f"Saved {len(records)} processed lungs and labels.csv to {OUT_BASE}")

zip_path = "/kaggle/working/processed_lungs.zip"
with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for root, _, files in os.walk(OUT_BASE):
        for fname in files:
            full = os.path.join(root, fname)
            # write with relative path inside the zip
            rel  = os.path.relpath(full, os.path.dirname(OUT_BASE))
            zf.write(full, rel)

print(f"Created ZIP archive at {zip_path}")

Saved 112116 processed lungs and labels.csv to /kaggle/working/processed_lungs
Created ZIP archive at /kaggle/working/processed_lungs.zip


In [19]:
from IPython.display import FileLink
FileLink("processed_lungs.zip")

/kaggle/working/processed_lungs.zip